# Twitter Sentiment Analysis

In [1]:
import nltk
import pandas as pd
tweets=pd.read_csv('Tweets.csv')
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


## Data Pre Processing

In [2]:
tweets_df= tweets.drop(tweets[tweets['airline_sentiment_confidence']<0.5].index,axis=0)
tweets_df.shape

(14404, 15)

In [3]:
X= tweets_df['text']
Y=tweets_df['airline_sentiment']

## Cleaning Data

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()

In [6]:
import re
cleaned_data=[]
for i in range(len(X)):
    tweet=re.sub('^a-zA-Z','',X.iloc[i])
    tweet=tweet.lower().split()
    tweet=[stemmer.stem(word) for word in tweet if(word not in stop_words) and (word not in punct)]
    tweet=' '.join(tweet)
    cleaned_data.append(tweet)

In [7]:
cleaned_data

['@virginamerica @dhepburn said.',
 '@virginamerica today... must mean need take anoth trip!',
 '@virginamerica realli aggress blast obnoxi "entertainment" guests\' face &amp; littl recours',
 '@virginamerica realli big bad thing',
 '@virginamerica serious would pay $30 flight seat playing. realli bad thing fli va',
 '@virginamerica yes, nearli everi time fli vx “ear worm” won’t go away :)',
 '@virginamerica realli miss prime opportun men without hat parody, there. https://t.co/mwpg7grezp',
 "@virginamerica well, didn't…but do! :-d",
 '@virginamerica amazing, arriv hour early. good me.',
 '@virginamerica know suicid second lead caus death among teen 10-24',
 '@virginamerica &lt;3 pretti graphics. much better minim iconography. :d',
 '@virginamerica great deal! alreadi think 2nd trip @australia &amp; even gone 1st trip yet! ;p',
 "@virginamerica @virginmedia i'm fli #fabul #seduct sky again! u take #stress away travel http://t.co/ahlxhhkiyn",
 '@virginamerica thanks!',
 '@virginamerica 

In [8]:
Y

0         neutral
2         neutral
3        negative
4        negative
5        negative
           ...   
14634    negative
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14404, dtype: object

In [9]:
sentiment_ordering=['negative','neutral','positive']
Y=Y.apply(lambda X: sentiment_ordering.index(X))

In [10]:
Y.head()

0    1
2    1
3    0
4    0
5    0
Name: airline_sentiment, dtype: int64

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000,stop_words=['virginamerica','unit'])
X_fin=cv.fit_transform(cleaned_data).toarray()
X_fin.shape

(14404, 3000)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model=MultinomialNB()

In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(X_fin,Y,test_size=0.3)
model.fit(X_train,Y_train)

MultinomialNB()

In [14]:
Y_pred=model.predict(X_test)


In [15]:
from sklearn.metrics import classification_report
cf= classification_report(Y_test,Y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.83      0.87      0.85      2767
           1       0.58      0.50      0.54       871
           2       0.70      0.67      0.69       684

    accuracy                           0.77      4322
   macro avg       0.70      0.68      0.69      4322
weighted avg       0.76      0.77      0.76      4322

